This is a notebook in which we will investigate whether a non-positional error model in deblur makes sense for short reads. It will consist of the following steps:

### (On a dataset)
1. Run deblur without hashing or trimming
2. Convert q2-deblur artifact to BIOM table
3. Trim using Table.collapse
4. On the same dataset, run deblur with trimming
5. Compare results of 3 and 4

Then, repeat steps 1-5 on many datsets on many lenghts and plot/examine results. Datasets to be pulled from mock community data

In [1]:
from qiime2.plugins import demux
from qiime2 import Artifact
from qiime2 import MetadataCategory
import qiime2

DATASET_ROOT = "/Users/massoudmaher/Documents/Code/mockrobiota/data"
MP_PATH = "/Users/massoudmaher/Documents/Code/qiime2-moving-pictures-tutorial"

single_end_seq = Artifact.load(MP_PATH + "/emp-single-end-sequences.qza")
barcodes = MetadataCategory.load("")
demuxed = demux.methods.emp_single(single_end_seq, )

TypeError: Visualizer callable cannot return anything. Its return annotation must be `None`, not <class 'pandas.core.frame.DataFrame'>. Write output to `output_dir`.

In [11]:
from qiime2.plugins.demux.methods import emp_single

In [12]:
emp_single?

In [14]:
from qiime2 import MetadataCategory

In [19]:
%%bash
# DOWNLOAD the seqs
source activate qiime2-2017.11 # this should be name of your qiime conda env
MOCK_DIR="/Users/massoudmaher/Documents/Code/mockrobiota/data/mock-3"
cd $MOCK_DIR
FORWARD_URL=`awk '$1 =="raw-data-url-forward-read" {print $2}' dataset-metadata.tsv`
REVERSE_URL=`awk '$1 =="raw-data-url-reverse-read" {print $2}' dataset-metadata.tsv`
BARCODE_URL=`awk '$1 =="raw-data-url-index-read" {print $2}' dataset-metadata.tsv`

# NOTE this download will take a while. Might want to run them all before
wget $FORWARD_URL
if [ "$REVERSE_URL" != "NA" ]; then
    wget $REVERSE_URL
fi

if [ "$BARCODE_URL" != "NA" ]; then
    wget $BARCODE_URL
fi

--2018-01-23 11:13:43--  https://s3-us-west-2.amazonaws.com/mockrobiota/latest/mock-3/mock-index-read.fastq.gz
Resolving s3-us-west-2.amazonaws.com... 52.218.144.52
Connecting to s3-us-west-2.amazonaws.com|52.218.144.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 365550 (357K) [application/x-gzip]
Saving to: 'mock-index-read.fastq.gz'

     0K .......... .......... .......... .......... .......... 14%  424K 1s
    50K .......... .......... .......... .......... .......... 28%  896K 0s
   100K .......... .......... .......... .......... .......... 42%  764K 0s
   150K .......... .......... .......... .......... .......... 56% 3.44M 0s
   200K .......... .......... .......... .......... .......... 70%  972K 0s
   250K .......... .......... .......... .......... .......... 84% 3.40M 0s
   300K .......... .......... .......... .......... .......... 98% 1.24M 0s
   350K ......                                                100% 66.9M=0.4s

2018-01-23 11:13:44 (

In [25]:
%%bash
# Prepare to run the pipeline 
source activate qiime2-2017.11 # this should be name of your qiime conda env
MOCK_DIR="/Users/massoudmaher/Documents/Code/mockrobiota/data/mock-3"
cd $MOCK_DIR

FORWARD_FP="$MOCK_DIR/source/mock-forward-read.fastq.gz"
REVERSE_FP="$MOCK_DIR/source/mock-reverse-read.fastq.gz"
BARCODE_FP="$MOCK_DIR/source/mock-index-read.fastq.gz"

#mkdir emp-paired-end-sequences
mv $FORWARD_FP emp-paired-end-sequences/forward.fastq.gz
mv $REVERSE_FP emp-paired-end-sequences/reverse.fastq.gz
mv $BARCODE_FP emp-paired-end-sequences/barcodes.fastq.gz

In [8]:
%%bash
source activate qiime2-2017.11 # this should be name of your qiime conda env
MOCK_DIR="/Users/massoudmaher/Documents/Code/mockrobiota/data/mock-3"
cd $MOCK_DIR

# TODO make this conditional on if it is paired or not
qiime tools import \
  --type EMPPairedEndSequences \
  --input-path emp-paired-end-sequences \
  --output-path emp-paired-end-sequences.qza

# Also import as single end
mkdir emp-single-end-sequences
## TODO must copy seqs and barcodes as sequences.fastq.gz and barcodes.fastq.gz
qiime tools import \
  --type EMPSingleEndSequences \
  --input-path emp-single-end-sequences \
  --output-path emp-single-end-sequences.qza

In [9]:
%%bash
source activate qiime2-2017.11 # this should be name of your qiime conda env
MOCK_DIR="/Users/massoudmaher/Documents/Code/mockrobiota/data/mock-3"
cd $MOCK_DIR

## TODO make this conditional on if it is paired or not
qiime demux emp-paired \
  --i-seqs emp-paired-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-category BarcodeSequence \
  --o-per-sample-sequences demux.qza
  
qiime demux emp-single \
  --i-seqs emp-single-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-category BarcodeSequence \
  --o-per-sample-sequences demux_single.qza

Saved SampleData[SequencesWithQuality] to: demux_single.qza


In [15]:
%%bash
# Case 1: We trim BEFORE deblurring (with deblur)
source activate qiime2-2017.11 # this should be name of your qiime conda env
MOCK_DIR="/Users/massoudmaher/Documents/Code/mockrobiota/data/mock-3"
cd $MOCK_DIR

qiime quality-filter q-score \
 --i-demux demux.qza \
 --o-filtered-sequences demux-filtered.qza \
 --o-filter-stats demux-filter-stats.qza

Plugin error from quality-filter:

  Argument to parameter 'demux' is not a subtype of SampleData[SequencesWithQuality].

Debug info has been saved to /var/folders/9m/dy7kzyp50fldn0ydz_9wj8180000gn/T/qiime2-q2cli-err-q5eqnna2.log


In [17]:
from qiime2.plugins.quality_filter.methods import q_score

In [20]:
#qiime deblur denoise-16S \
#  --i-demultiplexed-seqs demux-filtered.qza \
#  --p-trim-length 120 \
#  --o-representative-sequences rep-seqs-deblur.qza \
#  --o-table table-deblur.qza \
#  --p-sample-stats \
#  --o-stats deblur-stats.qza

from qiime2.plugins.deblur.methods import denoise_16S

## Starting over and doing it with qiime artifact api
Also doing it for mock-6 now since it is forward only

In [ ]:
%%bash
# DOWNLOAD the seqs
#source activate qiime2-2017.11 # this should be name of your qiime conda env
MOCK_DIR="/Users/massoudmaher/Documents/Code/mockrobiota/data/mock-6"
cd $MOCK_DIR
FORWARD_URL=`awk '$1 =="raw-data-url-forward-read" {print $2}' dataset-metadata.tsv`
REVERSE_URL=`awk '$1 =="raw-data-url-reverse-read" {print $2}' dataset-metadata.tsv`
BARCODE_URL=`awk '$1 =="raw-data-url-index-read" {print $2}' dataset-metadata.tsv`

# NOTE this download will take a while. Might want to run them all before
wget $FORWARD_URL
if [ "$REVERSE_URL" != "NA" ]; then
    wget $REVERSE_URL
fi

if [ "$BARCODE_URL" != "NA" ]; then
    wget $BARCODE_URL
fi

In [21]:
%%bash
# rename the seqs
MOCK_DIR="/Users/massoudmaher/Documents/Code/mockrobiota/data/mock-6"
cd $MOCK_DIR
# NOTE this assumes we are only doing forward reads and no paired
# If we are doing paired we need to make the names different
mv mock-forward-read.fastq.gz sequences.fastq.gz
mv mock-index-read.fastq.gz barcodes.fastq.gz

In [28]:
%%bash
# Put the seqs in their own directory so Artifact.import_data works
MOCK_DIR="/Users/massoudmaher/Documents/Code/mockrobiota/data/mock-6"
cd $MOCK_DIR
mkdir emp-single-end-sequences
mv sequences.fastq.gz emp-single-end-sequences/
mv barcodes.fastq.gz emp-single-end-sequences/

In [46]:
# import seqs as qiime artifact
from qiime2 import Artifact
MOCK_DIR="/Users/massoudmaher/Documents/Code/mockrobiota/data/mock-6"
# NOTE for paired end first param is different
mock6_seqs = Artifact.import_data("EMPSingleEndSequences", MOCK_DIR +"/emp-single-end-sequences")

In [47]:
# demux seqs
# NOTE that this is different for paired end reads
from qiime2.plugins.demux.methods import emp_single
from qiime2 import Metadata 

mock6_barcode_metadata = Metadata.load(MOCK_DIR + "/sample-metadata.tsv").get_category("BarcodeSequence")
mock6_demux, = emp_single(mock6_seqs, mock6_barcode_metadata) # NOTE the , in assignment so output is artifact and not tuple

KeyboardInterrupt: 

In [67]:
mock6_demux

<artifact: SampleData[SequencesWithQuality] uuid: 29bb6f74-d1f9-433d-b53e-9da250c52fd3>

In [68]:
## quality filter + deblur
from qiime2.plugins.quality_filter.methods import q_score

mock6_demux_filtered, mock6_demux_filter_stats = q_score(mock6_demux)

In [77]:
from qiime2.plugins.deblur.methods import denoise_16S
NUM_CORES = 4

# TODO BIG TODO-- figure out what length to trim for each mock dataset
mock6_deblur_100nt, mock6_rep_seqs, mock6_deblur_stats = \
    denoise_16S(mock6_demux_filtered, 100, 
                jobs_to_start = NUM_CORES, hashed_feature_ids = False)

In [81]:
# Trim reads AFTER deblur (we do not always do this)
import biom
mock6_biom = mock6_deblur_100nt.view(biom.Table)
mock6_biom_trim_75 = mock6_biom.collapse(lambda i, m: i[:75], axis='observation')

# Import trimmed table back into qiime2 artifact
mock6_deblur_trim_75 = Artifact.import_data("FeatureTable[Frequency]", mock6_trim_75);

In [ ]:
# Testing whether wget package or calling wget is faster because downloading is major  factor in making
# runtime longer
import subprocess
import time

def download_mock_data(working_dir_fp, sequences_url, barcodes_url, metadata_url):
    """
    Downloads data-- helper method
    
    :param working_dir_fp: filepath where sequences_url + barcodes_url file are downloaded to 
                           and put into a directory "emp-single-end-sequences". Ideally, this should be a
                           mock-<n> directory from when you clone the mockrobiota github repo
                           should not end with "/"
    :param sequences_url: URL from which sequences are downloaded from using wget
    :param barcodes_url: URL from which barcodes are downloaded from using wget
    :param metadata_url: URL from which we get metadata file
    """
    try:
        #print("Downloading " + sequences_url)
        #subprocess.call(["wget", sequences_url], cwd=working_dir_fp)
        #print("Downloading " + barcodes_url)
        #subprocess.call(["wget", barcodes_url], cwd=working_dir_fp)
        print("Downloading " + metadata_url)
        subprocess.call(["wget", metadata_url], cwd=working_dir_fp)
    except OSError:
        print("Failed to download")
        return None

import wget
def download_mock_data_wget(working_dir_fp, sequences_url, barcodes_url, metadata_url):
    """
    Downloads data-- helper method
    
    :param working_dir_fp: filepath where sequences_url + barcodes_url file are downloaded to 
                           and put into a directory "emp-single-end-sequences". Ideally, this should be a
                           mock-<n> directory from when you clone the mockrobiota github repo
                           should not end with "/"
    :param sequences_url: URL from which sequences are downloaded from using wget
    :param barcodes_url: URL from which barcodes are downloaded from using wget
    :param metadata_url: URL from which we get metadata file
    """
    try:
        #print("Downloading " + sequences_url)
        #subprocess.call(["wget", sequences_url], cwd=working_dir_fp)
        #print("Downloading " + barcodes_url)
        #subprocess.call(["wget", barcodes_url], cwd=working_dir_fp)
        print("Downloading " + metadata_url)
        wget.download(metadata_url, working_dir_fp)
    except OSError:
        print("Failed to download")
        return None
    
# Test import_mock_dataset
working_dir_fp = "/Users/massoudmaher/Documents/Code/download_mock_data-testing"
# TODO get these programatically
seq_url="https://s3-us-west-2.amazonaws.com/mockrobiota/latest/mock-6/mock-forward-read.fastq.gz"
barcode_url="https://s3-us-west-2.amazonaws.com/mockrobiota/latest/mock-6/mock-index-read.fastq.gz"
metadata_url="https://raw.githubusercontent.com/caporaso-lab/mockrobiota/master/data/mock-6/sample-metadata.tsv"

start = time.time()
download_mock_data(working_dir_fp, seq_url, barcode_url, metadata_url)
end = time.time()
print("Time using subprocess {:f}".format(end-start))

start = time.time()
download_mock_data_wget(working_dir_fp, seq_url, barcode_url, metadata_url)
end = time.time()
print("Time using wget {:f}".format(end-start))

In [14]:
from qiime2 import Artifact
from qiime2.plugins.demux.methods import emp_single
from qiime2 import Metadata 
import os
import wget

# TODO get rid of magic strings
def import_mock_dataset(working_dir_fp, sequences_url, barcodes_url, metadata_url, metadata_barcode_column):
    """
    Downloads seqs, imports them as qiime artifact, demuxes them
    NOTE: the mockrobiota file/naming structure is very consistent so we take advatage of that
    if it changes, much of this will break

    :param working_dir_fp: filepath where sequences_url + barcodes_url file are downloaded to 
                           and put into a directory "emp-single-end-sequences". Ideally, this should be a
                           mock-<n> directory from when you clone the mockrobiota github repo
                           should not end with "/"
    :param sequences_url: URL from which sequences are downloaded from using wget
    :param barcodes_url: URL from which barcodes are downloaded from using wget
    :param metadata_url: URL from which we get metadata file
    :param metadata_barcode_column: column header in metadatafile that holds barcode data
    
    :return tuple of demuxed seqs artifact of type <TODO>, loaded metadata <TODO>, None if fails
    """
    print("Downloading " + sequences_url)
    wget.download(sequences_url, working_dir_fp)
    print("Downloading " + barcodes_url)
    wget.download(barcodes_url, working_dir_fp)
    print("Downloading " + metadata_url)
    wget.download(metadata_url, working_dir_fp)
    
    # setup files so qiime can work with them
    try:
        subprocess.call(["mkdir" ,"emp-single-end-sequences"], cwd=working_dir_fp)
    except OSError:
        # Probably just because dir already exists
        print("mkdir emp-single-end-sequences failed, probably a non-issue")
        
    seq_fn = os.path.basename(sequences_url)
    barcode_fn = os.path.basename(barcodes_url)
    try:
        # TODO this is different if we are doing paired reads
        subprocess.call(["mv", seq_fn, "emp-single-end-sequences/sequences.fastq.gz"], cwd=working_dir_fp)
        subprocess.call(["mv", barcode_fn, "emp-single-end-sequences/barcodes.fastq.gz"], cwd=working_dir_fp)
    except OSError:
        print("Failed to rename downloaded files")
        return None
    
    # do the import
    # TODO different if we are doing paired reads
    print("Importing seq data")
    seqs = Artifact.import_data("EMPSingleEndSequences", working_dir_fp + "/emp-single-end-sequences")
    
    # import metadata and demux the data
    print("Loading metadata")
    barcode_metadata = Metadata.load(working_dir_fp + "/sample-metadata.tsv").get_category("BarcodeSequence")
    print("Demuxing")
    demux, = emp_single(seqs, barcode_metadata) # NOTE the , in assignment so output is artifact and not tuple
    return demux, barcode_metadata

In [15]:
# Test import_mock_dataset
working_dir_fp = "/Users/massoudmaher/Documents/Code/deblur-testing"
metadata_barcode_column="BarcodeSequence"
# TODO get these programatically
seq_url="https://s3-us-west-2.amazonaws.com/mockrobiota/latest/mock-6/mock-forward-read.fastq.gz"
barcode_url="https://s3-us-west-2.amazonaws.com/mockrobiota/latest/mock-6/mock-index-read.fastq.gz"
metadata_url="https://raw.githubusercontent.com/caporaso-lab/mockrobiota/master/data/mock-6/sample-metadata.tsv"

mock6_demux, mock6_barcode_metadata = \
    import_mock_dataset(working_dir_fp, seq_url, barcode_url, metadata_url, metadata_barcode_column)

Importing seq data
Loading metadata
Demuxing


In [16]:
print(type(mock6_demux))
print(type(mock6_barcode_metadata))

<class 'qiime2.sdk.result.Artifact'>
<class 'qiime2.metadata.MetadataCategory'>


In [52]:
from qiime2.plugins.quality_filter.methods import q_score
from qiime2.plugins.deblur.methods import denoise_16S
import biom

# TODO type enforcements
def establish_dataset(demuxed_seqs, pre_trim_length, post_trim_length = None):
    """
    Given demuxed sequences, deblurs them and returns the result

    :param demuxed_data: qiime artifact of demuxed data
    :param pre_trim_length: length that we want to trim sequences to before deblur is ran
    :param post_trim_length: length we want to trimg sequences to after delbur is ran
    
    :return FeatureTable[Frequency] of deblured data. Post-trimmed to length
    post_trim_length if post_trim_length is not None
    """
    print("Quality filtering (with default params)")
    demuxed_qfiltered, demuxed_qf_stats = q_score(demuxed_seqs)
    
    print("Deblur-ing with trim length {:d}".format(pre_trim_length))
    # TODO use jobs_to_start = NUM_CORES to speed up?
    deblurred, repseq, deblur_stats = \
        denoise_16S(demuxed_qfiltered, pre_trim_length, hashed_feature_ids = False)
    
    # if post_trim is not None, perform the collapse
    if(post_trim_length != None):
        print("Trimming post-demuxed seqs to {:d}".format(post_trim_length))
        demuxed_biom = deblurred.view(biom.Table)
        post_trimmed_biom = \
            demuxed_biom.collapse(lambda i, m:i[:post_trim_length], axis="observation")
        
        post_trimmed = Artifact.import_data("FeatureTable[Frequency]", post_trimmed_biom);
        return post_trimmed
    
    return deblurred

In [24]:
mock6_deblurred_100 = establish_dataset(mock6_demux, 100)

Quality filtering (with default params)
Deblur-ing with trim length 100


In [53]:
mock6_deblurred_75 = establish_dataset(mock6_demux, 100, post_trim_length = 75)

Quality filtering (with default params)
Deblur-ing with trim length 100
Trimming post-demuxed seqs to 75


In [69]:
import pandas as pd

dataset_metadata_url = "https://raw.githubusercontent.com/caporaso-lab/mockrobiota/master/data/mock-6/dataset-metadata.tsv"
metadata_fp = "/Users/massoudmaher/Documents/Code/deblur-testing/dataset-metadata.tsv"
working_dir_fp = "/Users/massoudmaher/Documents/Code/deblur-testing"
def get_dl_urls(dataset_metadata_url, metadata_fp, working_dir_fp):
    """
    :param dataset_metadata_url is url where we can download dataset metadata like this
    https://github.com/caporaso-lab/mockrobiota/blob/master/data/mock-6/dataset-metadata.tsv
    :param metadata_fp is filepath to sample-metadata.tsv file found in mock-<n> dir
    :param working_dir_fp: filepath where sequences_url + barcodes_url file are downloaded to 
                           and put into a directory "emp-single-end-sequences". Ideally, this should be a
                           mock-<n> directory from when you clone the mockrobiota github repo
                           should not end with "/"
    :return tuple of 2 strings, URLs of sequence and index files respectively
    """
    wget.download(dataset_metadata_url, working_dir_fp)
    metadata = pd.read_table(metadata_fp, index_col = 0)
    
    seq_url = metadata.loc["raw-data-url-forward-read", "value"] 
    barcode_url = metadata.loc["raw-data-url-index-read", "value"] 
    
    return seq_url, barcode_url

print(get_dl_urls(dataset_metadata_url, metadata_fp, working_dir_fp))

('https://s3-us-west-2.amazonaws.com/mockrobiota/latest/mock-6/mock-forward-read.fastq.gz', 'https://s3-us-west-2.amazonaws.com/mockrobiota/latest/mock-6/mock-index-read.fastq.gz')


In [3]:
from qiime2.plugins.deblur.methods import denoise_16S
denoise_16S?